## Preparazione dataset Simpsons

1) Conversione a file

In [83]:
import pandas as pd

dataset = "../Dataset/simpsons_orig.csv"

full = pd.read_csv(dataset)
full.columns

/tmp/ipykernel_619/3908780766.py:5: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  full = pd.read_csv(dataset)


Index(['id', 'episode_id', 'number', 'raw_text', 'timestamp_in_ms',
       'speaking_line', 'character_id', 'location_id', 'raw_character_text',
       'raw_location_text', 'spoken_words', 'normalized_text', 'word_count'],
      dtype='object')

In [84]:
spoken = full[full['speaking_line'] == True].dropna().reset_index()
spoken.head(3)

,index,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,0,9549,32,209,"Miss Hoover: No, actually, it was a little of ...",848000,True,464.0,3.0,Miss Hoover,Springfield Elementary School,"No, actually, it was a little of both. Sometim...",no actually it was a little of both sometimes ...,31
1,1,9550,32,210,Lisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?,856000,True,9.0,3.0,Lisa Simpson,Springfield Elementary School,Where's Mr. Bergstrom?,wheres mr bergstrom,3
2,2,9551,32,211,Miss Hoover: I don't know. Although I'd sure l...,856000,True,464.0,3.0,Miss Hoover,Springfield Elementary School,I don't know. Although I'd sure like to talk t...,i dont know although id sure like to talk to h...,22


In [85]:
episodes = spoken['episode_id']
locations = spoken['raw_location_text']
said_sentences = spoken['raw_text']

In [86]:
print(len(episodes), len(locations), len(said_sentences))

108680 108680 108680


In [87]:
sentences = []

for i in range(len(episodes)):
    sent = f"{episodes[i]}: {locations[i]}: {said_sentences[i]}"
    sentences.append(sent)

sentences[0:10]

["32: Springfield Elementary School: Miss Hoover: No, actually, it was a little of both. Sometimes when a disease is in all the magazines and all the news shows, it's only natural that you think you have it.",
 "32: Springfield Elementary School: Lisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?",
 "32: Springfield Elementary School: Miss Hoover: I don't know. Although I'd sure like to talk to him. He didn't touch my lesson plan. What did he teach you?",
 '32: Springfield Elementary School: Lisa Simpson: That life is worth living.',
 "32: Springfield Elementary School: Edna Krabappel-Flanders: The polls will be open from now until the end of recess. Now, (SOUR) just in case any of you have decided to put any thought into this, we'll have our final statements. Martin?",
 "32: Springfield Elementary School: Martin Prince: (HOARSE WHISPER) I don't think there's anything left to say.",
 '32: Springfield Elementary School: Edna Krabappel-Flanders: Bart?',
 '32: Springfield Elementary School

In [88]:
print(sentences[0])

32: Springfield Elementary School: Miss Hoover: No, actually, it was a little of both. Sometimes when a disease is in all the magazines and all the news shows, it's only natural that you think you have it.


In [89]:
import re

new_sentences = []

for sent in sentences:
    newsent = re.sub(r"\(.*\)", "", sent)
    new_sentences.append(newsent)

new_sentences[0:10]

["32: Springfield Elementary School: Miss Hoover: No, actually, it was a little of both. Sometimes when a disease is in all the magazines and all the news shows, it's only natural that you think you have it.",
 "32: Springfield Elementary School: Lisa Simpson:  Where's Mr. Bergstrom?",
 "32: Springfield Elementary School: Miss Hoover: I don't know. Although I'd sure like to talk to him. He didn't touch my lesson plan. What did he teach you?",
 '32: Springfield Elementary School: Lisa Simpson: That life is worth living.',
 "32: Springfield Elementary School: Edna Krabappel-Flanders: The polls will be open from now until the end of recess. Now,  just in case any of you have decided to put any thought into this, we'll have our final statements. Martin?",
 "32: Springfield Elementary School: Martin Prince:  I don't think there's anything left to say.",
 '32: Springfield Elementary School: Edna Krabappel-Flanders: Bart?',
 '32: Springfield Elementary School: Bart Simpson: Victory party unde

Export txt, sents only:

In [90]:
outfile = open("../out/simpsons.txt", 'w')

for sent in new_sentences:
    outfile.write(f"{sent}\n")

outfile.close()

Process sents to JSON:

In [93]:
import json

simpsons = {}

#add progressive id as key:
for i in range(len(new_sentences)):
    splitted = new_sentences[i].strip().split(':')
    if len(splitted) == 4:
        simpsons[i] = {'episode_id': splitted[0].strip(), 'location': splitted[1].strip(), 'character': splitted[2].strip(), 'sentence': splitted[3].strip()}

json.dump(simpsons, open("../out/simpsons.json", 'w'))

Process sents to DataFrame:

In [94]:
episode_id = []
location = []
character = []
sentence = []

for sent in new_sentences:
    splitted = sent.strip().split(':')
    if len(splitted) == 4:
        episode_id.append(splitted[0].strip())
        location.append(splitted[1].strip())
        character.append(splitted[2].strip())
        sentence.append(splitted[3].strip())

df_simpsons = pd.DataFrame({'episode_id': episode_id, 'location': location, 'character': character, 'sentence': sentence})
df_simpsons.to_csv('../out/simpsons.csv', index=False)